In [2]:
#Check Python Version
import shutil
import sys
sys.version

'3.8.2 (default, Jan 31 2023, 18:34:03) \n[GCC 12.2.0]'

In [3]:
#Upgrade pip
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [4]:
#Check TF & TFX Versioning
import tensorflow as tf
print(tf.__version__)
from tfx import v1 as tfx
print(tfx.__version__)


2.11.0
1.12.0


In [5]:
#Setup Variables as examplegen_playground
import os

# Pipeline name
PIPELINE_NAME = "importer_node_playground"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = './artifacts'
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Folder path to data
DATA_ROOT = './data/'

# Global variable for ExampleGen component
EXAMPLE_GEN = tfx.components.CsvExampleGen


from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [ ]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

# Importer Node
examples_importer = tfx.dsl.Importer(
    source_uri='../artifacts/CopyExampleGen/examples/2/1/',
    artifact_type=tfx.types.standard_artifacts.Examples).with_id(
        'examples_importer')

context.run(examples_importer)

In [7]:
###################################################################################
### [USE CASE 1]
### Tfrecords are already created but not in Examples Artifact form

### This component will:
### 1. Accept a dict with with {'split_name': './path/to/split_name/tfrecords.gz'}
### 2. Add them to a folder to follow Example Artifact directory structure
### 3. Import them with Importer node to register the external resource into MLMD
###################################################################################
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Create pipeline to run Importer node
def _create_pipeline(
  pipeline_name: str,
  pipeline_root: str,
  data_root: str,
  metadata_path: str
  ) -> tfx.dsl.Pipeline:

  context = InteractiveContext()
  ### Get external source data
  # Source directory of external Examples Artifacts
  source_examples_artifact_uri = '../artifacts/'

  # Destination directory for source
  destination_examples_artifact_uri = './artifacts/'

  # Get all files from source_examples_artifact_uri
  files = os.listdir(source_examples_artifact_uri)

  # Import source files to destination
  shutil.copytree(source_examples_artifact_uri, destination_examples_artifact_uri, dirs_exist_ok=True)


  # example_gen = tfx.components.CsvExampleGen(input_base=data_root)
  # print(example_gen.outputs['examples'])

  # Importer Node
  examples_importer = tfx.dsl.Importer(
      source_uri='../artifacts/CopyExampleGen/examples/2/1/',
      artifact_type=tfx.types.standard_artifacts.Examples).with_id(
          'examples_importer')

  context.run(examples_importer)
  print("IMPORTER_NODE: ", examples_importer.outputs)
  print("IMPORTER_NODE: ", examples_importer.outputs['result'].additional_properties.keys)
  print(list(examples_importer.outputs['result'].additional_properties.keys()))


  # Test if downstream component accepts output from Importer Node
  statistics_gen = tfx.components.StatisticsGen(
      examples=examples_importer.outputs['result'])


  # EXAMPLEGEN COMPONENT OUTPUT
  # Brings data into the pipeline.
  # example_gen = tfx.components.CsvExampleGen(input_base=data_root)
  # print("EXAMPLE_GEN: ", example_gen.outputs['examples'])

  # print("tfx.components.CsvExampleGen.outputs['examples'] output: \n\n",
  #   "Artifact Type: ", example_gen.outputs['examples']._artifact_type, "\n"
  #   "Producer Component Id: ", example_gen.outputs['examples'].producer_component_id, "\n"
  #   "Output Key: ", example_gen.outputs['examples'].output_key, "\n"
  #   "Additional Properties: ", example_gen.outputs['examples'].additional_properties, "\n"
  #   "Additional Custom Properties: ", example_gen.outputs['examples'].additional_custom_properties, "\n"
  # )
  # print(list(example_gen.outputs['examples'].additional_properties.keys()))

  components = [
    # example_gen,
    examples_importer,
    statistics_gen
  ]

  return tfx.dsl.Pipeline(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
    components=components
)

In [9]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=METADATA_PATH)
  )

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Using deployment config:
 executor_specs {
  key: "StatisticsGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.statistics_gen.executor.Executor"
      }
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "metadata/importer_node_playground/metadata.db"
      connection_mode: READWRITE_OPENCREATE
    }
  }
}

INFO:absl:Using connection config:
 sqlite {
  filename_uri: "metadata/importer_node_playground/metadata.db"
  connection_mode: READWRITE_OPENCREATE
}

INFO:absl:Component examples_importer is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.dsl.components.common.importer.Importer"
  }
  id: "examples_importer"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "importer_node_playground"
      }
    }
  }
 

IMPORTER_NODE:  {'result': OutputChannel(artifact_type=Examples, producer_component_id=examples_importer, output_key=result, additional_properties={}, additional_custom_properties={})}
IMPORTER_NODE:  <built-in method keys of dict object at 0x7f3af98eb100>
[]


ValueError: No splits for examples artifact: Artifact(artifact: id: 18
type_id: 14
uri: "../artifacts/CopyExampleGen/examples/2/1/"
custom_properties {
  key: "is_external"
  value {
    int_value: 1
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.12.0"
  }
}
state: LIVE
create_time_since_epoch: 1676047493360
last_update_time_since_epoch: 1676047493360
, artifact_type: id: 14
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
base_type: DATASET
)

In [ ]:
# [SPLIT_TRAIN] View Dataset Artifact as tf.record
train_uri = os.path.join('./artifacts/CopyExampleGen/examples/1/', 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset:
  # Prints out tf.record
  print(tfrecord)
  
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()

  # Prints out parsed tfrecord as JSON
  example.ParseFromString(serialized_example)
  print(example)


In [ ]:
# [SPLIT_EVAL] View Dataset Artifact
eval_uri = os.path.join('./artifacts/CsvExampleGen/examples/1/', 'Split-eval')

tfrecord_filenames = [os.path.join(eval_uri, name)
                      for name in os.listdir(eval_uri)]

print(tfrecord_filenames)


dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(3):
  # Prints out tf.record
  print(tfrecord)

  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  
  # Prints out parsed tfrecord as JSON
  example.ParseFromString(serialized_example)
  print(example)